<a href="https://colab.research.google.com/github/npto04/PySpark/blob/main/PySpark_aula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando PySpark

In [ ]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 59.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=3bff2f3a586ec91440e472301769a8eb8be78f5ff85686fe9222f85882843755
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


# Importando bibliotecas principais do PySpark

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
import pyspark.sql.functions as F

# Criando conexão do PySpark

In [ ]:
spark = (SparkSession.builder
        .master("local")
        .appName("aula-pyspark")
        .config('spark.ui.port', '4050')
        .getOrCreate())

In [ ]:
spark

# Criando dataframe

In [ ]:
#CRIANDO DATAFRAME NO PYSPARK
esquema = ["nome", "cidade", "estado", "salario"]
dados = [
        ("João da Silva", "São Paulo", "SP", 1200.00),
        ("Maria Ribeiro", "Rio de Janeiro", "RJ", 2400.00),
        ("Pedro José", "Fortaleza", "CE", 4230.00),
        ("Carlos Victor Pereira", "Florianópolis", "SC", 1200.00) 
]
df = spark.createDataFrame(data=dados,schema=esquema)

In [ ]:
df.show() #mostra o dataframe

+--------------------+--------------+------+-------+
|                nome|        cidade|estado|salario|
+--------------------+--------------+------+-------+
|       João da Silva|     São Paulo|    SP| 1200.0|
|       Maria Ribeiro|Rio de Janeiro|    RJ| 2400.0|
|          Pedro José|     Fortaleza|    CE| 4230.0|
|Carlos Victor Per...| Florianópolis|    SC| 1200.0|
+--------------------+--------------+------+-------+



In [ ]:
#Retorna as informações do schema do DataFrame
df.printSchema()

root
 |-- nome: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- salario: double (nullable = true)



# Importando dataset para um dataframe

In [ ]:
dataset_path = "/content/drive/MyDrive/Datasets/arquivo_geral.csv"

In [ ]:
df2 = (
    spark
        .read
        .option("header", "true")
        .option("delimiter", ";")
        .option("inferschema", "true")
        .format("csv")
        .load(dataset_path)
)

In [ ]:
df2.show()

+------+------+----------+----------+---------------+-----------+----------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+-----------+----------------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|
| Norte|    RO|2020-02-05|         0|              0|          0|               0|
| Norte|    RO|2020-02-06|         0|              0|          0|               0|
| Norte|    RO|2020-02-07|         0|              0|          0|               0|
| No

In [ ]:
df2.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data: string (nullable = true)
 |-- casosNovos: integer (nullable = true)
 |-- casosAcumulados: integer (nullable = true)
 |-- obitosNovos: integer (nullable = true)
 |-- obitosAcumulados: integer (nullable = true)



In [ ]:
df2.toPandas()

,regiao,estado,data,casosNovos,casosAcumulados,obitosNovos,obitosAcumulados
0,Norte,RO,2020-01-30,0,0,0,0
1,Norte,RO,2020-01-31,0,0,0,0
2,Norte,RO,2020-02-01,0,0,0,0
3,Norte,RO,2020-02-02,0,0,0,0
4,Norte,RO,2020-02-03,0,0,0,0
...,...,...,...,...,...,...,...
2344,Centro-Oeste,DF,2020-04-21,9,881,0,24
2345,Centro-Oeste,DF,2020-04-22,65,946,1,25
2346,Centro-Oeste,DF,2020-04-23,17,963,0,25
2347,Centro-Oeste,DF,2020-04-24,26,989,1,26


# Ler dataset do Google Cloud Storage

## Configurar a sessão para usar GCS connector

install google cloud storage filesystem

In [ ]:
!pip install gcsfs

To begin, you will need a JSON keyfile so the connector can authenticate to Google Cloud Storage. You can follow [these directions](https://cloud.google.com/storage/docs/authentication#service_accounts) to obtain a JSON keyfile.

Once you have the JSON keyfile, you can configure framework that you use to use GCS connector when accessing data on Google Cloud Storage.

In [ ]:
path_keys = "/content/engdados-soulcode-ae874e15f2ec.json"

In [ ]:
gcs_conector = '/content/drive/MyDrive/GCS_connector/gcs-connector-hadoop2-latest.jar'

In [ ]:
spark = (SparkSession.builder.
        master("local").
        appName("netflix").
        config('spark.jars', gcs_conector).
        config('spark.hadoop.fs.gs.impl','com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem').
        config('spark.hadoop.fs.gs.auth.service.account.enable', 'true').
        config("google.cloud.auth.service.account.json.keyfile",path_keys).
        config('spark.hadoop.fs.gs.project.id','engdados-soulcode').
        getOrCreate())

In [ ]:
spark

## Ler dados do bucket na gcp

In [ ]:
file_path = 'gs://datasets_pyspark/arquivo_geral.csv'


df = spark.read.format('csv').load(file_path,sep=";",header=True)

In [ ]:
df.show()

+------+------+----------+----------+---------------+-----------+----------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+-----------+----------------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|
| Norte|    RO|2020-02-05|         0|              0|          0|               0|
| Norte|    RO|2020-02-06|         0|              0|          0|               0|
| Norte|    RO|2020-02-07|         0|              0|          0|               0|
| No

# Comando SELECT

In [ ]:
df2.select("regiao","estado","casosNovos").show()

+------+------+----------+
|regiao|estado|casosNovos|
+------+------+----------+
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
+------+------+----------+
only showing top 20 rows



In [ ]:
regiao = F.col("regiao")
estado = F.col("estado")
casosNovos = F.col("casosNovos")
df2.select(regiao,estado,casosNovos).show()

+------+------+----------+
|regiao|estado|casosNovos|
+------+------+----------+
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
| Norte|    RO|         0|
+------+------+----------+
only showing top 20 rows



# Filtros

In [ ]:
#EXIBIR A REGIÃO, ESTADO E ÓBITOS ACUMULADOS DA REGIÃO SUDESTE

df2.select(F.col("regiao"), F.col("estado"), F.col("obitosAcumulados")).filter(F.col("regiao") == "Sudeste").show(20)

+-------+------+----------------+
| regiao|estado|obitosAcumulados|
+-------+------+----------------+
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
|Sudeste|    MG|               0|
+-------+------+----------------+
only showing top 20 rows



In [ ]:
df2.filter(F.col("estado") == "SP").show()

+-------+------+----------+----------+---------------+-----------+----------------+
| regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+-------+------+----------+----------+---------------+-----------+----------------+
|Sudeste|    SP|2020-01-30|         0|              0|          0|               0|
|Sudeste|    SP|2020-01-31|         0|              0|          0|               0|
|Sudeste|    SP|2020-02-01|         0|              0|          0|               0|
|Sudeste|    SP|2020-02-02|         0|              0|          0|               0|
|Sudeste|    SP|2020-02-03|         0|              0|          0|               0|
|Sudeste|    SP|2020-02-04|         0|              0|          0|               0|
|Sudeste|    SP|2020-02-05|         0|              0|          0|               0|
|Sudeste|    SP|2020-02-06|         0|              0|          0|               0|
|Sudeste|    SP|2020-02-07|         0|              0|          0|          

In [ ]:
#1ª FORMA - TRABALHANDO COM FILTER ENCADEADO E O AND
df2.filter(F.col("estado") == "MG").filter(F.col("casosNovos") > 10).show(100)

+-------+------+----------+----------+---------------+-----------+----------------+
| regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+-------+------+----------+----------+---------------+-----------+----------------+
|Sudeste|    MG|2020-03-19|        14|             29|          0|               0|
|Sudeste|    MG|2020-03-22|        45|             83|          0|               0|
|Sudeste|    MG|2020-03-23|        45|            128|          0|               0|
|Sudeste|    MG|2020-03-26|        20|            153|          0|               0|
|Sudeste|    MG|2020-03-27|        36|            189|          0|               0|
|Sudeste|    MG|2020-03-28|        16|            205|          0|               0|
|Sudeste|    MG|2020-03-29|        26|            231|          0|               0|
|Sudeste|    MG|2020-03-30|        30|            261|          1|               1|
|Sudeste|    MG|2020-03-31|        14|            275|          1|          

In [ ]:
#2ª FORMA - TRABALHANDO COM FILTER ENCADEADO E O AND
df2.filter("regiao = 'Sudeste' and casosNovos > 30").show(50)

+-------+------+----------+----------+---------------+-----------+----------------+
| regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+-------+------+----------+----------+---------------+-----------+----------------+
|Sudeste|    MG|2020-03-22|        45|             83|          0|               0|
|Sudeste|    MG|2020-03-23|        45|            128|          0|               0|
|Sudeste|    MG|2020-03-27|        36|            189|          0|               0|
|Sudeste|    MG|2020-04-01|        39|            314|          1|               3|
|Sudeste|    MG|2020-04-02|        56|            370|          1|               4|
|Sudeste|    MG|2020-04-04|        33|            430|          0|               6|
|Sudeste|    MG|2020-04-05|        68|            498|          0|               6|
|Sudeste|    MG|2020-04-07|        34|            559|          2|              11|
|Sudeste|    MG|2020-04-08|        55|            614|          3|          

In [ ]:
#3ª FORMA - TRABALHANDO COM FILTER ENCADEADO E O AND
df2.filter((F.col("regiao") == "Sudeste") & (F.col("casosNovos") > 20)).show(30)

+-------+------+----------+----------+---------------+-----------+----------------+
| regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+-------+------+----------+----------+---------------+-----------+----------------+
|Sudeste|    MG|2020-03-22|        45|             83|          0|               0|
|Sudeste|    MG|2020-03-23|        45|            128|          0|               0|
|Sudeste|    MG|2020-03-27|        36|            189|          0|               0|
|Sudeste|    MG|2020-03-29|        26|            231|          0|               0|
|Sudeste|    MG|2020-03-30|        30|            261|          1|               1|
|Sudeste|    MG|2020-04-01|        39|            314|          1|               3|
|Sudeste|    MG|2020-04-02|        56|            370|          1|               4|
|Sudeste|    MG|2020-04-03|        27|            397|          2|               6|
|Sudeste|    MG|2020-04-04|        33|            430|          0|          

In [ ]:
#1ª FORMA - TRABALHANDO COM FILTER ENCADEADO E O OR
df2.filter("regiao = 'Sudeste' or regiao = 'Norte'").show(200)

+------+------+----------+----------+---------------+-----------+----------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+-----------+----------------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|
| Norte|    RO|2020-02-05|         0|              0|          0|               0|
| Norte|    RO|2020-02-06|         0|              0|          0|               0|
| Norte|    RO|2020-02-07|         0|              0|          0|               0|
| No

In [ ]:
#2ª FORMA - TRABALHANDO COM FILTER ENCADEADO E O OR
df2.filter((F.col("regiao") == "Sudeste") | (F.col("regiao") == "Norte")).show(30)

+------+------+----------+----------+---------------+-----------+----------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+-----------+----------------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|
| Norte|    RO|2020-02-05|         0|              0|          0|               0|
| Norte|    RO|2020-02-06|         0|              0|          0|               0|
| Norte|    RO|2020-02-07|         0|              0|          0|               0|
| No

In [ ]:
filtro = F.col("regiao") == "Sudeste"

df2.filter(filtro).show(10)

+-------+------+----------+----------+---------------+-----------+----------------+
| regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+-------+------+----------+----------+---------------+-----------+----------------+
|Sudeste|    MG|2020-01-30|         0|              0|          0|               0|
|Sudeste|    MG|2020-01-31|         0|              0|          0|               0|
|Sudeste|    MG|2020-02-01|         0|              0|          0|               0|
|Sudeste|    MG|2020-02-02|         0|              0|          0|               0|
|Sudeste|    MG|2020-02-03|         0|              0|          0|               0|
|Sudeste|    MG|2020-02-04|         0|              0|          0|               0|
|Sudeste|    MG|2020-02-05|         0|              0|          0|               0|
|Sudeste|    MG|2020-02-06|         0|              0|          0|               0|
|Sudeste|    MG|2020-02-07|         0|              0|          0|          

In [ ]:
colunas = ['regiao', 'casosNovos']
df2.select(colunas).show(5)

+------+----------+
|regiao|casosNovos|
+------+----------+
| Norte|         0|
| Norte|         0|
| Norte|         0|
| Norte|         0|
| Norte|         0|
+------+----------+
only showing top 5 rows



# WHERE

In [ ]:
df2.where(F.col("regiao") == 'Nordeste').show(10)

+--------+------+----------+----------+---------------+-----------+----------------+
|  regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+--------+------+----------+----------+---------------+-----------+----------------+
|Nordeste|    MA|2020-01-30|         0|              0|          0|               0|
|Nordeste|    MA|2020-01-31|         0|              0|          0|               0|
|Nordeste|    MA|2020-02-01|         0|              0|          0|               0|
|Nordeste|    MA|2020-02-02|         0|              0|          0|               0|
|Nordeste|    MA|2020-02-03|         0|              0|          0|               0|
|Nordeste|    MA|2020-02-04|         0|              0|          0|               0|
|Nordeste|    MA|2020-02-05|         0|              0|          0|               0|
|Nordeste|    MA|2020-02-06|         0|              0|          0|               0|
|Nordeste|    MA|2020-02-07|         0|              0|          

In [ ]:
#MOSTRAR APENAS OS ESTADOS DA REGIÃO NORTE E QUE COMECEM COM A LETRA A
df2.where(F.col("regiao") == "Norte").filter("estado LIKE '%A%'").show(20)

+------+------+----------+----------+---------------+-----------+----------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+-----------+----------------+
| Norte|    AC|2020-01-30|         0|              0|          0|               0|
| Norte|    AC|2020-01-31|         0|              0|          0|               0|
| Norte|    AC|2020-02-01|         0|              0|          0|               0|
| Norte|    AC|2020-02-02|         0|              0|          0|               0|
| Norte|    AC|2020-02-03|         0|              0|          0|               0|
| Norte|    AC|2020-02-04|         0|              0|          0|               0|
| Norte|    AC|2020-02-05|         0|              0|          0|               0|
| Norte|    AC|2020-02-06|         0|              0|          0|               0|
| Norte|    AC|2020-02-07|         0|              0|          0|               0|
| No

In [ ]:
#FILTRO POR ESTADOS QUE COMECEM COM A LETRA A
df2.filter(F.col("estado").startswith("A")).show(10)
 
#FILTRO POR ESTADOS QUE CONTENHAM A LETRA A 
df2.filter(F.col("estado").contains("A")).show(10)

#FILTRO POR ESTADOS QUE TERMINEM COM A LETRA A
df2.filter(F.col("estado").endswith("A")).show(10)


+------+------+----------+----------+---------------+-----------+----------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+-----------+----------------+
| Norte|    AC|2020-01-30|         0|              0|          0|               0|
| Norte|    AC|2020-01-31|         0|              0|          0|               0|
| Norte|    AC|2020-02-01|         0|              0|          0|               0|
| Norte|    AC|2020-02-02|         0|              0|          0|               0|
| Norte|    AC|2020-02-03|         0|              0|          0|               0|
| Norte|    AC|2020-02-04|         0|              0|          0|               0|
| Norte|    AC|2020-02-05|         0|              0|          0|               0|
| Norte|    AC|2020-02-06|         0|              0|          0|               0|
| Norte|    AC|2020-02-07|         0|              0|          0|               0|
| No

In [ ]:
df2.filter(F.col("estado").like("A%")).show(20)

+------+------+----------+----------+---------------+-----------+----------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+-----------+----------------+
| Norte|    AC|2020-01-30|         0|              0|          0|               0|
| Norte|    AC|2020-01-31|         0|              0|          0|               0|
| Norte|    AC|2020-02-01|         0|              0|          0|               0|
| Norte|    AC|2020-02-02|         0|              0|          0|               0|
| Norte|    AC|2020-02-03|         0|              0|          0|               0|
| Norte|    AC|2020-02-04|         0|              0|          0|               0|
| Norte|    AC|2020-02-05|         0|              0|          0|               0|
| Norte|    AC|2020-02-06|         0|              0|          0|               0|
| Norte|    AC|2020-02-07|         0|              0|          0|               0|
| No

In [ ]:
#UTILIZANDO O FILTRO COM UMA LISTA DE REGIÕES COM A FUNÇÃO ISIN
lista_regiao = ['Norte', 'Sul']
df2.filter(F.col("regiao").isin(lista_regiao)).show(30)

+------+------+----------+----------+---------------+-----------+----------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+-----------+----------------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|
| Norte|    RO|2020-02-05|         0|              0|          0|               0|
| Norte|    RO|2020-02-06|         0|              0|          0|               0|
| Norte|    RO|2020-02-07|         0|              0|          0|               0|
| No

In [ ]:
#UTILIZANDO O FILTRO COM UMA LISTA DE REGIÕES
df2.filter("regiao in ('Norte', 'Sul')").show(30)

#ATIVIDADE

('id':1,'nome':'Bob', 'idade':45,'gen':'M')
('id':2,'nome':'Gloria', 'idade':43,'gen':'F')
('id':4,'nome':'Albert', 'idade':28,'gen':'M')
('id':5,'nome':'Laura', 'idade':33,'gen':'F')
('id':8,'nome':'Simone', 'idade':18,'gen':'T')
('id':12,'nome':'Marta', 'idade':45,'gen':'F')
('id':45,'nome':'Jairo', 'idade':82,'gen':'M')
('id':13,'nome':'Teste', 'idade':38,'gen':'T')

Criar um DataFrame a partir das informações acima. Em seguida, fazer a análise utilizando o PySpark.






In [ ]:
dados = [{'id':1,'nome':'Bob', 'idade':45,'gen':'M'},
{'id':2,'nome':'Gloria', 'idade':43,'gen':'F'},
{'id':4,'nome':'Albert', 'idade':28,'gen':'M'},
{'id':5,'nome':'Laura', 'idade':33,'gen':'F'},
{'id':8,'nome':'Simone', 'idade':18,'gen':'T'},
{'id':12,'nome':'Marta', 'idade':45,'gen':'F'},
{'id':45,'nome':'Jairo', 'idade':82,'gen':'M'},
{'id':13,'nome':'Teste', 'idade':38,'gen':'T'}]

In [ ]:
ativ = spark.createDataFrame(data=dados)

In [ ]:
ativ.show()

+---+---+-----+------+
|gen| id|idade|  nome|
+---+---+-----+------+
|  M|  1|   45|   Bob|
|  F|  2|   43|Gloria|
|  M|  4|   28|Albert|
|  F|  5|   33| Laura|
|  T|  8|   18|Simone|
|  F| 12|   45| Marta|
|  M| 45|   82| Jairo|
|  T| 13|   38| Teste|
+---+---+-----+------+



In [ ]:
ativ.printSchema()

root
 |-- gen: string (nullable = true)
 |-- id: long (nullable = true)
 |-- idade: long (nullable = true)
 |-- nome: string (nullable = true)



1) Verifique as pessoas com idade superior a 30 anos

In [ ]:
idades = F.col("idade")
filtro_idade_maior_30 = idades > 30

In [ ]:
ativ.filter(filtro_idade_maior_30).show()

+---+---+-----+------+
|gen| id|idade|  nome|
+---+---+-----+------+
|  M|  1|   45|   Bob|
|  F|  2|   43|Gloria|
|  F|  5|   33| Laura|
|  F| 12|   45| Marta|
|  M| 45|   82| Jairo|
|  T| 13|   38| Teste|
+---+---+-----+------+



2) Verificar apenas as pessoas do sexo F

In [ ]:
genero = F.col("gen")
filtro_gen_F = genero == 'F'

In [ ]:
ativ.where(filtro_gen_F).show()

+---+---+-----+------+
|gen| id|idade|  nome|
+---+---+-----+------+
|  F|  2|   43|Gloria|
|  F|  5|   33| Laura|
|  F| 12|   45| Marta|
+---+---+-----+------+



# withColumn

In [ ]:
#WithColumn cria uma nova coluna contendo o valor passado dentro da função f.lit()
numero = 10
df2 = df.withColumn("Nova_coluna", F.lit(numero))
df2.show(20)

+------+------+----------+----------+---------------+-----------+----------------+-----------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|Nova_coluna|
+------+------+----------+----------+---------------+-----------+----------------+-----------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|         10|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|         10|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|         10|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|         10|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|         10|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|         10|
| Norte|    RO|2020-02-05|         0|              0|          0|               0|         10|
| Norte|    RO|2020-02-06|         0|             

In [ ]:
# Se não apontar para nova variavel, mudança não é efetuada
df.withColumn("Nova_coluna", F.lit(100))

df.show()

+------+------+----------+----------+---------------+-----------+----------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+-----------+----------------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|
| Norte|    RO|2020-02-05|         0|              0|          0|               0|
| Norte|    RO|2020-02-06|         0|              0|          0|               0|
| Norte|    RO|2020-02-07|         0|              0|          0|               0|
| No

In [ ]:
#FUNÇÃO PARA TRANSFORMAR OS DADOS DE UMA COLUNA DE UM TIPO PARA OUTRO (cast)
df2.printSchema()

df3 = df2.select(F.col("Nova_coluna").cast("string"))

df3.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data: string (nullable = true)
 |-- casosNovos: integer (nullable = true)
 |-- casosAcumulados: integer (nullable = true)
 |-- obitosNovos: integer (nullable = true)
 |-- obitosAcumulados: integer (nullable = true)
 |-- Nova_coluna: integer (nullable = false)

root
 |-- Nova_coluna: string (nullable = false)



In [ ]:
#CRIAR UMA NOVA COLUNA COM A SOMA DE CASOS NOVOS + CASOS ACUMULADOS

df4 = df2.withColumn("soma_novos_acumulados", F.col("casosNovos") + F.col("casosAcumulados"))
df4.filter(F.col("casosNovos") > 10).show(10)

+------+------+----------+----------+---------------+-----------+----------------+-----------+---------------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|Nova_coluna|soma_novos_acumulados|
+------+------+----------+----------+---------------+-----------+----------------+-----------+---------------------+
| Norte|    RO|2020-04-14|        22|             64|          0|               2|         10|                   86|
| Norte|    RO|2020-04-17|        16|             92|          0|               3|         10|                  108|
| Norte|    RO|2020-04-18|        18|            110|          0|               3|         10|                  128|
| Norte|    RO|2020-04-19|        18|            128|          1|               4|         10|                  146|
| Norte|    RO|2020-04-20|        32|            160|          0|               4|         10|                  192|
| Norte|    RO|2020-04-21|        39|            199|          0

# WHEN-OTHERWISE

In [ ]:
spark = (SparkSession.builder
         .master("local[3]")
         .appName("when_otherwise")
         .config("spark.ui.port", "4050")
         .getOrCreate()
         )

In [ ]:
d3 = ( spark.read
            .format("csv")
            .option("header", "true")
            .option("inferschema", "true")
            .option("delimiter", ";")
            .load("/content/drive/MyDrive/Datasets/arquivo_geral.csv")
      
)

In [ ]:
df2 = (df.withColumn("Status", F.when(F.col("casosNovos") > 0, F.lit("Possui Casos Novos"))
                    .otherwise(F.lit("Não possui casos novos"))))

df2.show(1000)

In [ ]:
#CRIANDO UMA NOVA COLUNA COM BASE EM MAIS DE DUAS CONDIÇÕES
df3 = (df.withColumn("Status", F.when((F.col('casosNovos') >= 50) & (F.col('casosNovos') <= 100), F.lit("Entre 50 e 100"))
                                .when((F.col('casosNovos') > 100 ) & (F.col('casosNovos') <= 200), F.lit('Entre 101 e 200'))
                                .when((F.col('casosNovos') > 200 ) & (F.col('casosNovos') <= 300), F.lit('Entre 201 e 300'))
                                .otherwise(F.lit('Entre outra faixa de valores'))))

df3.show(20)

+------+------+----------+----------+---------------+-----------+----------------+--------------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|              Status|
+------+------+----------+----------+---------------+-----------+----------------+--------------------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|Entre outra faixa...|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|Entre outra faixa...|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|Entre outra faixa...|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|Entre outra faixa...|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|Entre outra faixa...|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|Entre outra faixa...|
| Norte|    RO|2020-02-05|         0|              0|          0

In [ ]:
#PERCORRENDO UMA SUBSTRING DENTRO DE UM DATAFRAME
df4 =( df3.withColumn("Dia", F.substring(F.col("data"), 9, 2))
          .withColumn("Mes", F.substring(F.col("data"), 6, 2))
          .withColumn("Ano", F.substring(F.col("data"), 1, 4))

)

# DIA F.substring(F.col("data"), 9, 2)
# MES F.substring(F.col("data"), 6, 2)
# ANO F.substring(F.col("data"), 1, 4)
df4.select(F.col("data"), F.col("Dia"), F.col("Mes"), F.col("Ano")).show(20)

+----------+---+---+----+
|      data|Dia|Mes| Ano|
+----------+---+---+----+
|2020-01-30| 30| 01|2020|
|2020-01-31| 31| 01|2020|
|2020-02-01| 01| 02|2020|
|2020-02-02| 02| 02|2020|
|2020-02-03| 03| 02|2020|
|2020-02-04| 04| 02|2020|
|2020-02-05| 05| 02|2020|
|2020-02-06| 06| 02|2020|
|2020-02-07| 07| 02|2020|
|2020-02-08| 08| 02|2020|
|2020-02-09| 09| 02|2020|
|2020-02-10| 10| 02|2020|
|2020-02-11| 11| 02|2020|
|2020-02-12| 12| 02|2020|
|2020-02-13| 13| 02|2020|
|2020-02-14| 14| 02|2020|
|2020-02-15| 15| 02|2020|
|2020-02-16| 16| 02|2020|
|2020-02-17| 17| 02|2020|
|2020-02-18| 18| 02|2020|
+----------+---+---+----+
only showing top 20 rows



In [ ]:
df5 =( df3.withColumn("Dia", F.substring(F.col("data"), 9, 2).cast("integer"))
          .withColumn("Mes", F.substring(F.col("data"), 6, 2).cast("integer"))
          .withColumn("Ano", F.substring(F.col("data"), 1, 4).cast("integer"))

)

df5.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data: string (nullable = true)
 |-- casosNovos: integer (nullable = true)
 |-- casosAcumulados: integer (nullable = true)
 |-- obitosNovos: integer (nullable = true)
 |-- obitosAcumulados: integer (nullable = true)
 |-- Status: string (nullable = false)
 |-- Dia: integer (nullable = true)
 |-- Mes: integer (nullable = true)
 |-- Ano: integer (nullable = true)



# FUNÇÕES PARA ORDENAÇÃO DE COLUNAS - ORDEM CRESCENTE OU DECRESCENTE SORT ou ORDERBY




In [ ]:

#df5.show(10)

df5.select(F.col("estado"), F.col('casosNovos'), F.col("Dia"), F.col("Mes"), F.col("Ano")).sort(F.col("Dia").desc(), F.col("Mes")).show(200)
df5.select(F.col("estado"), F.col('casosNovos'), F.col("Dia"), F.col("Mes"), F.col("Ano")).orderBy(F.col("Mes").asc(), F.col("Dia")).show(200)

# GroupBy

In [ ]:
#AGRUPAR OS RESULTADOS DE ACORDO COM UMA COLUNA ESPECÍFICA
#FAZER A SOMA DE CASOS NOVOS POR ESTADO

df5.groupBy(F.col("estado")).sum("casosNovos").show()

+------+---------------+
|estado|sum(casosNovos)|
+------+---------------+
|    SC|           1209|
|    RO|            328|
|    PI|            297|
|    AM|           3635|
|    RR|            345|
|    GO|            506|
|    TO|             50|
|    MT|            247|
|    SP|          20004|
|    PB|            447|
|    ES|           1595|
|    RS|           1096|
|    MS|            217|
|    AL|            501|
|    MG|           1481|
|    PA|           1579|
|    BA|           2081|
|    SE|            153|
|    PE|           4507|
|    CE|           5421|
+------+---------------+
only showing top 20 rows



In [ ]:
df5.show(5)

#QUANTIDADE DE ESTADOS POR REGIÃO NO DATAFRAME - COUNT()
df5.groupBy(df5.regiao).count().show()

+------+------+----------+----------+---------------+-----------+----------------+--------------------+---+---+----+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|              Status|Dia|Mes| Ano|
+------+------+----------+----------+---------------+-----------+----------------+--------------------+---+---+----+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|Entre outra faixa...| 30|  1|2020|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|Entre outra faixa...| 31|  1|2020|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|Entre outra faixa...|  1|  2|2020|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|Entre outra faixa...|  2|  2|2020|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|Entre outra faixa...|  3|  2|2020|
+------+------+----------+----------+---------------+-----------

In [ ]:
df5.groupBy(df5.estado).count().show()

+------+-----+
|estado|count|
+------+-----+
|    SC|   87|
|    RO|   87|
|    PI|   87|
|    AM|   87|
|    RR|   87|
|    GO|   87|
|    TO|   87|
|    MT|   87|
|    SP|   87|
|    PB|   87|
|    ES|   87|
|    RS|   87|
|    MS|   87|
|    AL|   87|
|    MG|   87|
|    PA|   87|
|    BA|   87|
|    SE|   87|
|    PE|   87|
|    CE|   87|
+------+-----+
only showing top 20 rows



In [ ]:
df5.select(df5.regiao, df5.estado).distinct().groupBy("regiao").count().show()

+------------+-----+
|      regiao|count|
+------------+-----+
|    Nordeste|    9|
|         Sul|    3|
|     Sudeste|    4|
|Centro-Oeste|    4|
|       Norte|    7|
+------------+-----+



In [ ]:
df5.groupBy("regiao", "estado").count().groupBy("regiao").count().show()

+------------+-----+
|      regiao|count|
+------------+-----+
|    Nordeste|    9|
|         Sul|    3|
|     Sudeste|    4|
|Centro-Oeste|    4|
|       Norte|    7|
+------------+-----+



# Funçao aggregate

In [ ]:
df5.show(1)

+------+------+----------+----------+---------------+-----------+----------------+--------------------+---+---+----+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|              Status|Dia|Mes| Ano|
+------+------+----------+----------+---------------+-----------+----------------+--------------------+---+---+----+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|Entre outra faixa...| 30|  1|2020|
+------+------+----------+----------+---------------+-----------+----------------+--------------------+---+---+----+
only showing top 1 row



In [ ]:
df7 = (df5.groupBy("regiao").
 agg(F.sum("casosAcumulados"),
    F.min("casosNovos"),
    F.max("casosNovos"),
     F.avg("casosNovos"),
     F.count_distinct("estado")).show())

+------------+--------------------+---------------+---------------+------------------+-------------+
|      regiao|sum(casosAcumulados)|min(casosNovos)|max(casosNovos)|   avg(casosNovos)|count(estado)|
+------------+--------------------+---------------+---------------+------------------+-------------+
|    Nordeste|              160011|              0|            688|20.808429118773947|            9|
|         Sul|               58308|              0|            192|13.199233716475096|            3|
|     Sudeste|              394635|              0|           2178| 85.94252873563218|            4|
|Centro-Oeste|               32840|              0|             65| 5.698275862068965|            4|
|       Norte|               65439|              0|            441| 11.29720853858785|            7|
+------------+--------------------+---------------+---------------+------------------+-------------+



In [ ]:
schema = ['nome','nota']
dados = [('João',8.0),('José',7.0),('Paula',10.0),('Patrícia', 9.5)]
df_agg = spark.createDataFrame(data=dados, schema=schema)
df_agg.show()

+--------+----+
|    nome|nota|
+--------+----+
|    João| 8.0|
|    José| 7.0|
|   Paula|10.0|
|Patrícia| 9.5|
+--------+----+



In [ ]:
(df_agg.agg(
    F.sum('nota').alias('soma_notas'),
    F.avg('nota').alias('media_notas')
    )).show()

+----------+-----------+
|soma_notas|media_notas|
+----------+-----------+
|      34.5|      8.625|
+----------+-----------+



# Struct type

Principais Tipos de Dados Básicos no Pyspark:

- ByteType (int)
- ShortType (int)
- IntegerType(int)
- LongType(int)

- FloatType(float)
- DoubleType(float)

- StringType(str)

- BooleanType(bool)



Tipos Complexos de Dados no Pyspark:
- BinaryType(bytearray)

- TimestampType(datetime.datetime)
- DateType(datetime.date)

- ArrayType(lista, tupla ou array)
- MapType(dicionario)

- StructType (lista ou tupla)
- StructField (Um valor correspondente ao tipo do campo)

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType

In [ ]:
spark = (SparkSession.builder
         .master("local")
         .appName("struct-type")
         .config("spark.ui.port", "4050")
         .getOrCreate()
         )

In [ ]:
esquema = (
    StructType([
        StructField("Regiao",StringType(), True),
        StructField("Estado", StringType(), True),
        StructField("Data", StringType(), True),
        StructField("casosNovos", IntegerType(), True),
        StructField("casosAcumulados", IntegerType(), True),
        StructField("obitosNovos", IntegerType(), True),
        StructField("obitosAcumulados", IntegerType(), True)
    ])
)

In [ ]:
df_struct = (
    spark
       .read
       .format("csv")
       .option("header", "true")
       .option("delimiter", ";")
       .load("/content/drive/MyDrive/Datasets/arquivo_geral.csv",schema=esquema)
)

In [ ]:
df_struct.printSchema()

root
 |-- Regiao: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Data: string (nullable = true)
 |-- casosNovos: integer (nullable = true)
 |-- casosAcumulados: integer (nullable = true)
 |-- obitosNovos: integer (nullable = true)
 |-- obitosAcumulados: integer (nullable = true)



In [ ]:
df_struct.show()

+------+------+----------+----------+---------------+-----------+----------------+
|Regiao|Estado|      Data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+-----------+----------------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|
| Norte|    RO|2020-02-05|         0|              0|          0|               0|
| Norte|    RO|2020-02-06|         0|              0|          0|               0|
| Norte|    RO|2020-02-07|         0|              0|          0|               0|
| No

# Manipulando arquivos (importar e exportar)

Lendo arquivos

In [ ]:
#1ª FORMA
df = ( spark.read
            .format('csv')
            .option('header', 'false')
            .option('delimiter', ';')
            .load('/content/drive/MyDrive/Datasets/arquivo_geral.csv', schema = schema)
)

In [ ]:
# 2ª FORMA
df = ( spark.read
            .format('csv')
            .option('header', 'false')
            .option('delimiter', ';')
            .schema(schema)
            .load('/content/drive/MyDrive/Datasets/arquivo_geral.csv')
)

In [ ]:
# 3ª FORMA
df = (
    spark.read
         .csv('/content/drive/MyDrive/Datasets/arquivo_geral.csv', sep=';', schema=schema )
)

In [ ]:
# 4ª FORMA

df = (
    spark.read
         .format('csv')
         .options(header = True, sep = ';')
         .schema(schema)
         .load('/content/drive/MyDrive/Datasets/arquivo_geral.csv')
)

In [ ]:
#LEITURA DE ARQUIVOS DO TIPO JSON
df_json = spark.read.format('json').load('caminho/arquivo.json')

In [ ]:
#LEITURA DE ARQUIVOS DO TIPO ORC
df_orc = spark.read.format('orc').load('caminho/arquivo.orc')

LENDO E ESCREVENDO PARQUET
- Armazenamento colunar, em contraste com o CSV, que armazena baseado nas linhas. Assim, quando uma Query é realizada é possível ignorar os dados não relevantes de maneira rápida e fácil resultando em operações mais eficientes. 
- Preservação de metadados, incluindo os tipos das colunas, o que garante eficiência e praticidade na escrita e leitura (Não é necessário especificar schemas para arquivos parquet)
- Suporte a dados estruturados de forma aninhada, como listas
- Otimizado para processar dados particionados com volume na casa dos gigabytes para cada arquivo
- Compressão de dados na escrita, de forma a ocupar menos espaço
- Integração com ferramentas como AWS Athena, Amazon Redshift Spectrum, Google BigQuery e Google Dataproc.


In [ ]:
#LEITURA DE ARQUIVOS DO TIPO PARQUET
df_parquet = spark.read.format('parquet').load('caminho/arquivo')

EXPORTAR OS ARQUIVOS NOS FORMATOS CSV, JSON, ORC, PARQUET (salvar)

In [ ]:
#CSV
df.write.format('csv').save('caminho/arquivo.csv')
#JSON
df.write.format('json').save('caminho/arquivo.json')
#ORC
df.write.format('orc').save('caminho/arquivo.orc')
#PARQUET
df.write.format('parquet').save('caminho/arquivo')

In [ ]:
# MODE:
# - append - Empilhar os arquivos aos já existentes
# - ignore - Retornar um erro
# - overwrite - Sobrescrever os dados já existentes
# error: (default) - Retornar o erro, dizendo que já existem dados
#Exemplo
df.write.format('parquet').mode('error').save('caminho/arquivo')

# Tipos de dados

Principais Valores que podem vir nas Colunas

Numéricos 

Strings

Datas

Nulos

Array


## Principais operações matemáticas com dados (métodos)

round(): Arredonda o valor numérico 

ceil(): Arredonda o valor numérico para o maior inteiro mais próximo 

floor(): Arredonda o valor numérico para o menor inteiro mais próximo 

sqrt(): retorna a raiz quadrada do valor exp(): Retorna a exponencial do valor 

log(): Retorna a logaritmo natural do valor 

log10(): Retorna a logaritmo na base 10 do valor 

greatest(): Retorna o maior valor dentre os valores da coluna. Análogo ao max() 

least(): Retorna o menor valor dentre os valores da coluna. Análogo ao min()

In [ ]:
#df.limit(5).toPandas()
#df.withColumn('maximo_casos_novos', F.least(F.col('casosNovos'), F.col('casosAcumulados'))).show(500)
df.withColumn('raiz_quadrada', F.sqrt(F.col('casosNovos'))).show(500)

## Operações com strings

upper(): Retorna o string em letras maiúsculas 

lower(): Retorna o string em letras minúsculas 

initcap(): Retorna a primeira letra de cada String em maiúsculas 

trim(): Retira o espaço em branco do início e fim da string 

ltrim() / rtrim() - retira os espaços em branco do início / fim respectivamente 

lpad() / rpad() - Acrescenta um caractere no início e fim, respectivamente, até que o string tenha um determinado comprimento 

length(): Retorna o comprimento do string em quantidade de caracteres 

split(): quebra o string a partir de um padrão e retorna o array com os strings resultantes 

concat(): concatena uma ou mais colunas de string 

concat_ws(): concatena uma ou mais colunas de string com um separador entre elas 

regexp_extract(): retorna um match no string a partir de um padrão regex 

regexp_replace(): substitui um match no string a partir de um padrão regex com outros caracteres 

substring(): retorna os caracteres do string que estão entre dois índices especificados. Análogo a F.col().substring()

In [ ]:
# df.withColumn('regiao', F.upper(F.col('regiao'))).show(5)
# df.withColumn('regiao', F.lower(F.col('regiao'))).show(5)
# df.withColumn('estado', F.initcap(F.col('estado'))).show(5)
# df.withColumn('regiao_estado', F.concat(F.col('regiao'), F.lit(','), F.col('estado'))).show(20)
# df.withColumn('regiao_estado', F.concat_ws('_', F.col('estado'), F.col('regiao'), F.col('data'))).show(20)
( df.withColumn('dia', F.substring(F.col('data'), 9, 2))
    .withColumn('mes', F.substring(F.col('data'), 6, 2))
    .withColumn('ano', F.substring(F.col('data'), 1, 4)).show(20)
)

## Operações com Datas

add_months(): retorna a data depois de adicionar x meses 

months_between(): retorna a diferença entre duas datas em meses. 

date_add(): retorna a data depois de adicionar "x" dias. 

date_sub(): retorna a data depois de subtrair "x" dias. 

next_day(): retorna o dia seguinte de alguma data. 

datediff(): retorna a diferença entre duas datas em dias. 

current_date(): retorna a data atual. 

dayofweek() / dayofmonth() / dayofyear(): retorna o dia relativo à semana, ao mês e ao ano, respectivamente. 

weekofyear(): retorna a semana relativa ao ano. 

second() / minute() / hour(): retorna os segundos, os minutos e as horas de uma coluna de datetime, respectivamente. 

month() year(): retorna o mês e o ano de uma coluna de data, respectivamente. 

last_day(): retorna o último dia do mês do qual a data considerada pertence. 

to_date(): transforma a coluna no tipo data (DateType()). 

trunc(): formata a data para a unidade especificada.

In [ ]:
# TIMESTAMP:  yyyy-MM-dd HH:mm:ss.SSSS  
# DATE:    yyyy-MM-dd

#CONVERTENDO UM CAMPO STRING PARA DATE
df3 = df2.withColumn('data', F.to_date(F.col('data'), 'yyyy-mm-dd'))
df2.printSchema()
df3.printSchema()
df3.show(10)

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data: string (nullable = true)
 |-- casosNovos: integer (nullable = true)
 |-- casosAcumulados: integer (nullable = true)
 |-- obitosNovos: integer (nullable = true)
 |-- obitosAcumulados: integer (nullable = true)

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data: date (nullable = true)
 |-- casosNovos: integer (nullable = true)
 |-- casosAcumulados: integer (nullable = true)
 |-- obitosNovos: integer (nullable = true)
 |-- obitosAcumulados: integer (nullable = true)

+------+------+----------+----------+---------------+-----------+----------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+-----------+----------------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|
| Norte|    RO|2020-01-31|         0|              0|          0| 

In [ ]:
#CONVERTENDO UM CAMPO TIMESTAMP PARA DATE

df_data = spark.createDataFrame(data=[(1, "2022-02-15 14:59:58.000")], schema=['id', 'data_completa'])

df_data.show(truncate=False)
df_data.printSchema()

df_data.withColumn('data_convertida', F.to_date(F.col('data_completa'))).show(truncate=False)

+---+-----------------------+
|id |data_completa          |
+---+-----------------------+
|1  |2022-02-15 14:59:58.000|
+---+-----------------------+

root
 |-- id: long (nullable = true)
 |-- data_completa: string (nullable = true)

+---+-----------------------+---------------+
|id |data_completa          |data_convertida|
+---+-----------------------+---------------+
|1  |2022-02-15 14:59:58.000|2022-02-15     |
+---+-----------------------+---------------+



In [ ]:
#Retornando o ANO com a função YEAR
#df2.printSchema()
( df2.withColumn('ano', F.year(F.col('data')))
     .withColumn('mes', F.month(F.col('data')))
     .withColumn('dia', F.dayofmonth(F.col('data')))
     .withColumn('semana', F.dayofweek(F.col('data')))
     .withColumn('semana', F.dayofyear(F.col('data')))
     .withColumn('data2', F.lit('2022-03-14'))
     .withColumn('dif_data', F.datediff(F.col('data2'), F.col('data')))
     .withColumn('dif_data_mes', F.months_between(F.col('data2'),F.col('data')))
     .withColumn('data_add', F.date_add(F.col('data2'), 60))
     .limit(10).toPandas()
)
# F.datediff('2022-03-14', '01-01-2022') #Retorna a diferença em dias de 2 datas
# F.months_between('2022-03-14', '01-01-2022') # Retorna a diferença em meses de 2 datas



,regiao,estado,data,casosNovos,casosAcumulados,obitosNovos,obitosAcumulados,ano,mes,dia,semana,data2,dif_data,dif_data_mes,data_add
0,Norte,RO,2020-01-30,0,0,0,0,2020,1,30,30,2022-03-14,774,25.483871,2022-05-13
1,Norte,RO,2020-01-31,0,0,0,0,2020,1,31,31,2022-03-14,773,25.451613,2022-05-13
2,Norte,RO,2020-02-01,0,0,0,0,2020,2,1,32,2022-03-14,772,25.419355,2022-05-13
3,Norte,RO,2020-02-02,0,0,0,0,2020,2,2,33,2022-03-14,771,25.387097,2022-05-13
4,Norte,RO,2020-02-03,0,0,0,0,2020,2,3,34,2022-03-14,770,25.354839,2022-05-13
5,Norte,RO,2020-02-04,0,0,0,0,2020,2,4,35,2022-03-14,769,25.322581,2022-05-13
6,Norte,RO,2020-02-05,0,0,0,0,2020,2,5,36,2022-03-14,768,25.290323,2022-05-13
7,Norte,RO,2020-02-06,0,0,0,0,2020,2,6,37,2022-03-14,767,25.258065,2022-05-13
8,Norte,RO,2020-02-07,0,0,0,0,2020,2,7,38,2022-03-14,766,25.225806,2022-05-13
9,Norte,RO,2020-02-08,0,0,0,0,2020,2,8,39,2022-03-14,765,25.193548,2022-05-13


## Nulos

drop(): retira do DataFrame as linhas com nulos, com base no que foi passado para o argumento how:

‒ any (default): retira todas as linhas com pelo menos um valor nulo nas colunas.

‒ all: somente retira as linhas com todos os valores nulos nas colunas.

fillna(): preenche os valores nulos no DataFrame com uma constante, passada pelo usuário.

replace(): substitui o valor (não somente os valores nulos) por algum outro passado pelo usuário.

Obs: todas as funções acima aceitam um argumento subset, usado para identificar em quais colunas deve ser aplicada a função.

In [ ]:
schema = ['nome', 'idade', 'nota'] 

dados = [('Otavio', 16 , None), ('Patricia', None, 8.0), ('Gustavo', None, None), ('Beatriz', None, 15.0), ('Carla', 17, 10.0), (None, None, None)]

df4 = spark.createDataFrame(data=dados, schema=schema)

df4.show()


+--------+-----+----+
|    nome|idade|nota|
+--------+-----+----+
|  Otavio|   16|null|
|Patricia| null| 8.0|
| Gustavo| null|null|
| Beatriz| null|15.0|
|   Carla|   17|10.0|
|    null| null|null|
+--------+-----+----+



In [ ]:
#REMOVER TODAS AS LINHAS QUE CONTENHAM DADOS NULOS
df4.na.drop().show()
df4.na.drop(how='any').show()

+-----+-----+----+
| nome|idade|nota|
+-----+-----+----+
|Carla|   17|10.0|
+-----+-----+----+

+-----+-----+----+
| nome|idade|nota|
+-----+-----+----+
|Carla|   17|10.0|
+-----+-----+----+



In [ ]:
#REMOVER APENAS LINHAS INTEIRAMENTE NULAS
df4.na.drop(how='all').show()

+--------+-----+----+
|    nome|idade|nota|
+--------+-----+----+
|  Otavio|   16|null|
|Patricia| null| 8.0|
| Gustavo| null|null|
| Beatriz| null|15.0|
|   Carla|   17|10.0|
+--------+-----+----+



In [ ]:
df4.na.drop(subset=['idade']).show()

+------+-----+----+
|  nome|idade|nota|
+------+-----+----+
|Otavio|   16|null|
| Carla|   17|10.0|
+------+-----+----+



In [ ]:
df4.na.drop(subset=['nota']).show()

+--------+-----+----+
|    nome|idade|nota|
+--------+-----+----+
|Patricia| null| 8.0|
| Beatriz| null|15.0|
|   Carla|   17|10.0|
+--------+-----+----+



In [ ]:
#SUBSTITUIR VALORES NULOS POR UMA CONSTANTE
df4.na.fill("").show()
df4.na.fill(0).show()
df4.na.fill(0, subset=['idade', 'nota']).show()

+--------+-----+----+
|    nome|idade|nota|
+--------+-----+----+
|  Otavio|   16|null|
|Patricia| null| 8.0|
| Gustavo| null|null|
| Beatriz| null|15.0|
|   Carla|   17|10.0|
|        | null|null|
+--------+-----+----+

+--------+-----+----+
|    nome|idade|nota|
+--------+-----+----+
|  Otavio|   16| 0.0|
|Patricia|    0| 8.0|
| Gustavo|    0| 0.0|
| Beatriz|    0|15.0|
|   Carla|   17|10.0|
|    null|    0| 0.0|
+--------+-----+----+

+--------+-----+----+
|    nome|idade|nota|
+--------+-----+----+
|  Otavio|   16| 0.0|
|Patricia|    0| 8.0|
| Gustavo|    0| 0.0|
| Beatriz|    0|15.0|
|   Carla|   17|10.0|
|    null|    0| 0.0|
+--------+-----+----+



In [ ]:
df4.na.fill(value={'idade':0,'nome':"",'nota':0}).show()

+--------+-----+----+
|    nome|idade|nota|
+--------+-----+----+
|  Otavio|   16| 0.0|
|Patricia|    0| 8.0|
| Gustavo|    0| 0.0|
| Beatriz|    0|15.0|
|   Carla|   17|10.0|
|        |    0| 0.0|
+--------+-----+----+



In [ ]:
df4.na.fill(10, subset=['nota']).show()
df4.fillna(0, subset=['idade', 'nota']).show()

+--------+-----+----+
|    nome|idade|nota|
+--------+-----+----+
|  Otavio|   16|  10|
|Patricia| null|   8|
| Gustavo| null|  10|
| Beatriz| null|  15|
|   Carla|   17|  10|
|    null| null|  10|
+--------+-----+----+

+--------+-----+----+
|    nome|idade|nota|
+--------+-----+----+
|  Otavio|   16|   0|
|Patricia|    0|   8|
| Gustavo|    0|   0|
| Beatriz|    0|  15|
|   Carla|   17|  10|
|    null|    0|   0|
+--------+-----+----+



# Window Function

row_number(): Número da linha 

rank(): Ranking sobre alguma partição 

percent_rank(): Retorna um percentil sobre alguma partição 

dense_rank(): Retorna um ranking de linhas ntile(): Retorna uma identificação 

cume_dist(): Retorna uma distribuição cumulativa

In [ ]:
from pyspark.sql.window import Window

In [ ]:
schema = ['nome', 'departamento', 'estado', 'salario']

dados = [('Anderson', 'vendas', 'SP', 9000),
         ('Kennedy', 'vendas', 'RJ', 4500),
         ('Luciana', 'vendas', 'SP', 4500),
         ('Marilia', 'vendas', 'SP', 1200),
         ('Gustavo', 'financeiro', 'AM', 8000),
         ('Pedro', 'financeiro', 'AM', 2750),
         ('Juliana', 'financeiro', 'MG', 3000),
         ('Leticia', 'financeiro', 'RJ', 7500),
         ('Oswaldo', 'marketing', 'RJ', 2450),
         ('Denis', 'marketing', 'MG', 1300)
         
        ]
df = spark.createDataFrame(data=dados, schema=schema)

In [ ]:
#Particionar minha janela a partir de uma coluna
w0 = Window.partitionBy(F.col('departamento')).orderBy('salario')

In [ ]:
#row_number
df.withColumn('row_number', F.row_number().over(w0)).show()

+--------+------------+------+-------+----------+
|    nome|departamento|estado|salario|row_number|
+--------+------------+------+-------+----------+
|   Pedro|  financeiro|    AM|   2750|         1|
| Juliana|  financeiro|    MG|   3000|         2|
| Leticia|  financeiro|    RJ|   7500|         3|
| Gustavo|  financeiro|    AM|   8000|         4|
|   Denis|   marketing|    MG|   1300|         1|
| Oswaldo|   marketing|    RJ|   2450|         2|
| Marilia|      vendas|    SP|   1200|         1|
| Kennedy|      vendas|    RJ|   4500|         2|
| Luciana|      vendas|    SP|   4500|         3|
|Anderson|      vendas|    SP|   9000|         4|
+--------+------------+------+-------+----------+



In [ ]:
#RANK
df.withColumn('rank', F.rank().over(w0)).show()

+--------+------------+------+-------+----+
|    nome|departamento|estado|salario|rank|
+--------+------------+------+-------+----+
|   Pedro|  financeiro|    AM|   2750|   1|
| Juliana|  financeiro|    MG|   3000|   2|
| Leticia|  financeiro|    RJ|   7500|   3|
| Gustavo|  financeiro|    AM|   8000|   4|
|   Denis|   marketing|    MG|   1300|   1|
| Oswaldo|   marketing|    RJ|   2450|   2|
| Marilia|      vendas|    SP|   1200|   1|
| Kennedy|      vendas|    RJ|   4500|   2|
| Luciana|      vendas|    SP|   4500|   2|
|Anderson|      vendas|    SP|   9000|   4|
+--------+------------+------+-------+----+



In [ ]:
#DENSE_RANK
df.withColumn('dense_rank', F.dense_rank().over(w0)).show()

+--------+------------+------+-------+----------+
|    nome|departamento|estado|salario|dense_rank|
+--------+------------+------+-------+----------+
|   Pedro|  financeiro|    AM|   2750|         1|
| Juliana|  financeiro|    MG|   3000|         2|
| Leticia|  financeiro|    RJ|   7500|         3|
| Gustavo|  financeiro|    AM|   8000|         4|
|   Denis|   marketing|    MG|   1300|         1|
| Oswaldo|   marketing|    RJ|   2450|         2|
| Marilia|      vendas|    SP|   1200|         1|
| Kennedy|      vendas|    RJ|   4500|         2|
| Luciana|      vendas|    SP|   4500|         2|
|Anderson|      vendas|    SP|   9000|         3|
+--------+------------+------+-------+----------+



In [ ]:
#lag 
df.withColumn('lag', F.lag("salario", 1).over(w0)).show()

+--------+------------+------+-------+----+
|    nome|departamento|estado|salario| lag|
+--------+------------+------+-------+----+
|   Pedro|  financeiro|    AM|   2750|null|
| Juliana|  financeiro|    MG|   3000|2750|
| Leticia|  financeiro|    RJ|   7500|3000|
| Gustavo|  financeiro|    AM|   8000|7500|
|   Denis|   marketing|    MG|   1300|null|
| Oswaldo|   marketing|    RJ|   2450|1300|
| Marilia|      vendas|    SP|   1200|null|
| Kennedy|      vendas|    RJ|   4500|1200|
| Luciana|      vendas|    SP|   4500|4500|
|Anderson|      vendas|    SP|   9000|4500|
+--------+------------+------+-------+----+



In [ ]:
#lead 
df.withColumn('lead', F.lead("salario", 1).over(w0)).show()

+--------+------------+------+-------+----+
|    nome|departamento|estado|salario|lead|
+--------+------------+------+-------+----+
|   Pedro|  financeiro|    AM|   2750|3000|
| Juliana|  financeiro|    MG|   3000|7500|
| Leticia|  financeiro|    RJ|   7500|8000|
| Gustavo|  financeiro|    AM|   8000|null|
|   Denis|   marketing|    MG|   1300|2450|
| Oswaldo|   marketing|    RJ|   2450|null|
| Marilia|      vendas|    SP|   1200|4500|
| Kennedy|      vendas|    RJ|   4500|4500|
| Luciana|      vendas|    SP|   4500|9000|
|Anderson|      vendas|    SP|   9000|null|
+--------+------------+------+-------+----+



# Joins

In [ ]:
schema = ['id', 'nome', 'salario']

dados = [
         (1, 'João', 2000),
         (2, 'Maria', 5000),
         (3, 'Pedro', 8300),
         (4, 'Gustavo', 1200),
         (5, 'Ana', 10000),
         (6, 'Vitoria', 7500),
         (7, 'Carlos', 4800),
         (8, 'Claudia', 4300),
         (9, 'Andre', 2400),
]
df1 = spark.createDataFrame(data=dados, schema=schema)

In [ ]:
schema = ['id', 'cidade', 'estado']

dados = [
         (1, 'São Paulo', 'SP'),
         (2, 'Curitiba', 'PR'),
         (3, 'Fortaleza', 'CE'),
         (4, 'Ribeirão Preto', 'SP'),
         (8, 'Belo Horizonte', 'MG'),
         (9, 'Goiania', 'GO'),
         (10, 'Sobral', 'CE'),
         (11, 'Salvador', 'BA'),
         (12, 'São Luis', 'MA'),
]
df2 = spark.createDataFrame(data=dados, schema=schema)

## INNER JOINS

In [ ]:
df_inner = df1.join(df2, on=['id'], how='inner')

df_inner.show()

+---+-------+-------+--------------+------+
| id|   nome|salario|        cidade|estado|
+---+-------+-------+--------------+------+
|  1|   João|   2000|     São Paulo|    SP|
|  2|  Maria|   5000|      Curitiba|    PR|
|  3|  Pedro|   8300|     Fortaleza|    CE|
|  4|Gustavo|   1200|Ribeirão Preto|    SP|
|  8|Claudia|   4300|Belo Horizonte|    MG|
|  9|  Andre|   2400|       Goiania|    GO|
+---+-------+-------+--------------+------+



## LEFT JOIN, LEFTOUTER, LEFT_OUTER

In [ ]:
df_left = df1.join(df2, on=['id'], how='left')

df_left.show()

+---+-------+-------+--------------+------+
| id|   nome|salario|        cidade|estado|
+---+-------+-------+--------------+------+
|  1|   João|   2000|     São Paulo|    SP|
|  2|  Maria|   5000|      Curitiba|    PR|
|  3|  Pedro|   8300|     Fortaleza|    CE|
|  4|Gustavo|   1200|Ribeirão Preto|    SP|
|  5|    Ana|  10000|          null|  null|
|  6|Vitoria|   7500|          null|  null|
|  7| Carlos|   4800|          null|  null|
|  8|Claudia|   4300|Belo Horizonte|    MG|
|  9|  Andre|   2400|       Goiania|    GO|
+---+-------+-------+--------------+------+



## RIGHT JOIN

In [ ]:
df_right = df1.join(df2, on=['id'], how='right')

df_right.show()

+---+-------+-------+--------------+------+
| id|   nome|salario|        cidade|estado|
+---+-------+-------+--------------+------+
|  1|   João|   2000|     São Paulo|    SP|
|  2|  Maria|   5000|      Curitiba|    PR|
|  3|  Pedro|   8300|     Fortaleza|    CE|
|  4|Gustavo|   1200|Ribeirão Preto|    SP|
|  8|Claudia|   4300|Belo Horizonte|    MG|
|  9|  Andre|   2400|       Goiania|    GO|
| 10|   null|   null|        Sobral|    CE|
| 11|   null|   null|      Salvador|    BA|
| 12|   null|   null|      São Luis|    MA|
+---+-------+-------+--------------+------+



## FULL, OUTER, FULLOUTER, FULL_OUTER

In [ ]:
df_full = df1.join(df2, on=['id'], how='full')

df_full.show()

+---+-------+-------+--------------+------+
| id|   nome|salario|        cidade|estado|
+---+-------+-------+--------------+------+
|  1|   João|   2000|     São Paulo|    SP|
|  2|  Maria|   5000|      Curitiba|    PR|
|  3|  Pedro|   8300|     Fortaleza|    CE|
|  4|Gustavo|   1200|Ribeirão Preto|    SP|
|  5|    Ana|  10000|          null|  null|
|  6|Vitoria|   7500|          null|  null|
|  7| Carlos|   4800|          null|  null|
|  8|Claudia|   4300|Belo Horizonte|    MG|
|  9|  Andre|   2400|       Goiania|    GO|
| 10|   null|   null|        Sobral|    CE|
| 11|   null|   null|      Salvador|    BA|
| 12|   null|   null|      São Luis|    MA|
+---+-------+-------+--------------+------+

